**2단계:**

- 유투브 AI 내용 검색 Agent 추가
    - 유투브 API 활용
    - 예시 입력 프롬프트 : AI 뉴스관련 영상을 알려줘
    - Agent를 통해서 유투브 API를 통해 뉴스 검색 후 검색 된 내용을 프롬프트에 넣어서 알려주기
        - streamlit 유투브 영상 화면 까지 같이 보여주기 (재생할 수 있는)
    - Agent는 유저의 쿼리를 LLM이 분석해서 특정 함수를 실행
    - **공식문서 참고해서 Agent내용 학습후 코드화 (베스트)**
        - Quick Start (가이드 코드)
        - Lanchain : 안배웠다하더라도 스스로 찾아서 할 수 있어야한다.
        - App

# 아이디어 정리
1. 

In [ ]:
import os
import googleapiclient.discovery
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from pydantic import BaseModel
from langchain_core.prompts import PromptTemplate

In [ ]:
class SearchResult(BaseModel):
    """
    
    """

In [ ]:
class SearchAgent:
    def __init__(self, openai_api_key, youtube_api_key, llm_model="gpt-4o"):
        self.openai_api_key = openai_api_key
        self.youtube_api_key = youtube_api_key
        self.llm_model = llm_model

    def analyze_query(self, user_query):
        """
        LLM을 사용하여 유저 쿼리를 분석하고 검색 키워드를 반환.
        """
        llm = ChatOpenAI(
            model=self.llm_model,
            temperature=0.1,
            api_key=self.openai_api_key,
        )
        
        messages=[
            (
                "system",
                "You are an assistant that extracts relevant keywords for searching."
            ),
            ("human", f"{user_query}")
        ]
        
        response = llm.invoke(messages)
        
        return response.content

    def search_youtube(self, query, max_results=5):
        """
        YouTube API를 사용하여 검색.
        """
        youtube = googleapiclient.discovery.build(
            "youtube", "v3", developerKey=self.youtube_api_key
        )

        request = youtube.search().list(
            part="snippet",
            q=query,
            type="video",
            maxResults=max_results
        )
        response = request.execute()

        results = [
            {
                "title": item["snippet"]["title"],
                "description": item["snippet"]["description"],
                "video_id": item["id"]["videoId"]
            }
            for item in response.get("items", [])
        ]
        return results

    def format_results_for_display(self, results):
        """
        검색 결과를 스트림릿에서 보여줄 수 있도록 포맷팅.
        """
        for result in results:
            print(f"### {result['title']}")
            #print(result['description'])
            video_url = f"https://www.youtube.com/watch?v={result['video_id']}"
            print(video_url)
            #st.video(video_url)

In [ ]:
def main():
    try:
        load_dotenv()
        
        # 환경 변수에서 OpenAI API 키를 불러오기
        openai_api_key = os.getenv("OPENAI_API_KEY")
        if not openai_api_key:
            print("OPENAI_API_KEY 환경 변수를 설정해주세요.")
            return
        
        # 환경 변수에서 YouTube API 키를 불러오기
        youtube_api_key = os.getenv("YOUTUBE_API_KEY")
        if not youtube_api_key:
            print("YOUTUBE_API_KEY 환경 변수를 설정해주세요.")
            return

        # 유저 입력 받기
        print("YouTube AI Search Agent")
        user_query = input("검색할 내용을 입력하세요 (예: AI 뉴스 관련 영상을 알려줘):")

        if user_query:
            # Agent 초기화
            agent = SearchAgent(openai_api_key, youtube_api_key)

            # 쿼리 분석
            print("LLM을 통해 입력 쿼리를 분석 중입니다...")
            search_keywords = agent.analyze_query(user_query)
            print(f"검색 키워드: {search_keywords}")

            # YouTube 검색
            print("YouTube에서 검색 중입니다...")
            search_results = agent.search_youtube(search_keywords)

            # 검색 결과 표시
            if search_results:
                print("검색 결과:")
                agent.format_results_for_display(search_results)
            else:
                print("검색 결과가 없습니다.")
    except KeyboardInterrupt:
        print("Shutting down process...")
    
    except Exception as e:
        print(f"Error occurred: {e}")

if __name__ == "__main__":
    main()

YouTube AI Search Agent
LLM을 통해 입력 쿼리를 분석 중입니다...
검색 키워드: - AI 뉴스
- 인공지능 뉴스
- AI 기술 동향
- 인공지능 최신 소식
- AI 발전
- AI 트렌드
- 인공지능 혁신
- AI 관련 뉴스
- AI 산업 뉴스
- 인공지능 기술 뉴스
YouTube에서 검색 중입니다...
검색 결과:
### &quot;일자리 뺏을 거냐&quot; AI에 묻자…정교함에 창조자 놀랐다 / SBS 8뉴스
https://www.youtube.com/watch?v=3R7ys0mEmaI
### 10년 뒤 세상을 바꿀 신기한 미래기술 TOP 10 | 하늘을 나는 택시, AI 로봇, 인공지능 CCTV, 메타버스, SKT부스, 월드IT쇼
https://www.youtube.com/watch?v=aqBL0YrRyQw
### AI(인공지능) 전쟁이 시작됐다!  유망 AI 기업은 국내 첫 AI ETF 출시 /  AI 산업 분석  / 한국경제TV
https://www.youtube.com/watch?v=TjMrHQaaqUQ
### [AI 비즈니스] 07. 인공지능 최근 개발 트렌드
https://www.youtube.com/watch?v=djn7DXEV4qU
### EOST2022 인공지능 기술동향과 미래전망
https://www.youtube.com/watch?v=4Fa_6RKyD0g
